# TensorFlow Lite Flatbuffer Manipulation Example

This notebook looks at how TFLite flatbuffers can be read, modified and written.

In [13]:
import flatbuffers

## Install flatc compiler

The flatc compiler converts model format stored in a text schema to Python accessor classes. This isn't available for download, so it needs to be built from source. The version must match the flatbuffer library on the system, in this notebook we use flatbuffers 1.12.0.

In [1]:
!curl -L "https://github.com/google/flatbuffers/archive/v1.12.0.zip" -o flatbuffers.zip
!unzip -q flatbuffers.zip
!mv flatbuffers-1.12.0 flatbuffers
%cd flatbuffers
!cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release -DFLATBUFFERS_BUILD_TESTS=OFF
!make -j 8
!cp flatc /usr/local/bin/
%cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   124  100   124    0     0    252      0 --:--:-- --:--:-- --:--:--   252
100 1463k    0 1463k    0     0  1062k      0 --:--:--  0:00:01 --:--:-- 2616k
/Users/lukehutton/Library/Mobile Documents/com~apple~CloudDocs/University/Harvard Tiny ML/deployment/flatbuffers
CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


-- The C compiler identification is AppleClang 13.0.0.13000029
-- The CXX compiler identification is AppleClang 13.0.0.13000029
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - d

## Generate Python Accessor Classes

Using the text schema, flatc created python code to read and write the data held inside a serialized Flatbuffer file.

In [5]:
!wget -nc https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema_v3.fbs
!flatc --python --gen-object-api schema_v3.fbs

--2021-10-02 12:28:03--  https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/schema/schema_v3.fbs
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8002 (7.8K) [text/plain]
Saving to: ‘schema_v3.fbs’

schema_v3.fbs       100%[===================>]   7.81K  --.-KB/s    in 0.001s  

2021-10-02 12:28:04 (6.38 MB/s) - ‘schema_v3.fbs’ saved [8002/8002]



## Save and load model

Demonstrates how to load data from a file and turn it into a `Model` python object. This can then be saved back to a file.

In [9]:
import sys
sys.path.append('tflite')
import Model

def load_model_from_file(model_filename):
    with open(model_filename, "rb") as file:
        buffer_data = file.read()
    model_obj = Model.Model.GetRootAsModel(buffer_data, 0)
    model = Model.ModelT.InitFromObj(model_obj)
    return model

def save_model_to_file(model, model_filename):
    builder = flatbuffers.Builder(1024)
    model_offset = model.Pack(builder)
    builder.Finish(model_offset, file_identifier=b'TFL3')
    model_data = builder.Output()
    with open(model_filename, 'wb') as out_file:
        out_file.write(model_data)

## Download example model
In order to test the Flatbuffer loading code.

In [10]:
!curl -O 'https://storage.googleapis.com/download.tensorflow.org/models/tflite/micro/speech_commands_model_2020_04_27.zip'
!unzip speech_commands_model_2020_04_27.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78855  100 78855    0     0   316k      0 --:--:-- --:--:-- --:--:--  315k
Archive:  speech_commands_model_2020_04_27.zip
replace speech_commands_model/speech_commands_model_float.tflite? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Load, modify and save model

Loads a float model, applies a change to the float weights and saves the model again.

In [14]:
import numpy as np

model = load_model_from_file('speech_commands_model/speech_commands_model_float.tflite')
for buffer in model.buffers:
    if buffer.data is not None and len(buffer.data) > 1024:
        original_weights = np.frombuffer(buffer.data, dtype=np.float32)
        altered_weights = np.round(original_weights * (1/0.02)) * 0.02
        buffer.data = altered_weights.tobytes()
save_model_to_file(model, 'speech_commands_model/speech_commands_model_modified.tflite')

## Evaluating the impact of those changes

To evaluate the impact of the changes, a test data set needs to be downloaded and some utility classes to read and convert the training data need to be created.

In [ ]:
sys.path.append('')